In [1]:
import pandas as pd
import numpy as np

In [12]:
excel_file_path = './data/dataTmp.xlsx'

# 엑셀 파일의 각 시트를 다른 변수에 저장
menu_info = pd.read_excel(excel_file_path, sheet_name='Sheet1', header = 0, index_col = 0)
user_info = pd.read_excel(excel_file_path, sheet_name='Sheet2', header = 0, index_col = 0)

# 각 시트 데이터 확인
print("Sheet1 데이터:")
print(menu_info.head())

print("\nSheet2 데이터:")
print(user_info.head())

Sheet1 데이터:
        바지락  면  파  다시마  멸치  국간장  진간장  고춧가루  떡  어묵  ...  닭고기  양상추  토마토  체다치즈  \
바지락칼국수    3  3  2    2   1    1    0     1  0   0  ...    0    0    0     0   
떡볶이       0  0  0    0   0    1    0     0  3   3  ...    0    0    0     0   
해물찜       0  0  0    0   0    1    0     0  0   0  ...    0    0    0     0   
초밥        0  0  0    0   0    1    0     0  0   2  ...    0    0    0     0   
돈가스       0  0  0    0   0    0    0     0  0   0  ...    0    0    0     0   

        모짜렐라치즈  케첩  마요네즈  양배추  참기름  다진마늘  
바지락칼국수       0   0     0    0    0     0  
떡볶이          0   0     0    0    0     0  
해물찜          0   0     0    0    0     0  
초밥           0   0     0    0    0     0  
돈가스          0   0     0    0    0     0  

[5 rows x 51 columns]

Sheet2 데이터:
       바지락칼국수  떡볶이  해물찜  초밥  돈가스  샤부샤부  김치찌개  고등어구이  햄버거  짜장면  치킨  삼겹살  돼지갈비
User1       1    0    0   1    0     0     0      0    1    0   0    0     0
User2       0    1    0   1    1     0     1      0    0    0   0

In [14]:
menu_info.arr = menu_info.to_numpy()
user_info.arr = user_info.to_numpy()

In [16]:
user_info.arr

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [17]:
# 실제 R 행렬과 예측 행렬의 오차를 구하는 함수
def calculate_rmse(R, P, Q, non_zeros):
    error = 0

    full_pred_matrix = np.dot(P, Q.T)

    # 여기서 non_zeros는 아래 함수에서 확인할 수 있다.
    x_non_zero_ind = [non_zeros[0] for non_zeros in non_zeros]
    y_non_zero_ind = [non_zeros[1] for non_zeros in non_zeros]

    # 원 행렬 R에서 0이 아닌 값들만 추출한다.
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    # 예측 행렬에서 원 행렬 R에서 0이 아닌 위치의 값들만 추출하여 저장한다.
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape

    np.random.seed(1)
    P = np.random.normal(scale=1.0/K, size=(num_users, K))
    Q = np.random.normal(scale=1.0/K, size=(num_items, K))

    # R>0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장한다.
    non_zeros = [ (i, j, R[i, j]) for i in range(num_users)
                  for j in range(num_items) if R[i, j] > 0 ]

    # SGD 기법으로 P, Q 매트릭스를 업데이트 함
    for step in range(steps):
        for i, j, r in non_zeros:
            # 잔차 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)

            # Regulation을 반영한 SGD 업데이터 적용
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])

        rmse = get_rmse(R, P, Q, non_zeros)
        if step % 10 == 0:
            print("iter step: {0}, rmse: {1:4f}".format(step, rmse))

    return P, Q

In [ ]:
P, Q = matrix_factorization(R, K=3)